In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/mercedesbenz-greener-manufacturing/train.csv')
test = pd.read_csv('/kaggle/input/mercedesbenz-greener-manufacturing/test.csv')

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
train.dtypes

In [ ]:
numeric_lst=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_cols = list(train.select_dtypes(include=numeric_lst).columns)
numerical_cols

In [ ]:
train_mi = train[numerical_cols]
train_mi.head()

In [ ]:
train_mi = train_mi.drop('ID', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_mi.drop(labels=['y'], axis=1),
    train_mi['y'],
    test_size=0.25,
    random_state=0)

In [ ]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X_train, y_train)
mutual_info

In [ ]:

mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mi_scores = mutual_info.sort_values(ascending=False)

In [ ]:
mi_scores.head(10)

In [ ]:
num_feat = ['X127', 'X314', 'X261', 'X313', 'X263', 'X136','X232', 'X29',
           'X279','X76']
train_num = train[num_feat]
train_num

In [ ]:
sns.kdeplot(mi_scores)

In [ ]:
cat_lst=['object']
cat_cols = list(train.select_dtypes(include=cat_lst).columns)
cat_cols

In [ ]:
train_mi_cat = train[cat_cols]

In [ ]:
for col in train_mi_cat.columns:
    train_mi_cat[col],_ = train_mi_cat[col].factorize()

In [ ]:
train_mi_cat['y'] = train['y']
train_mi_cat.head()

In [ ]:
df_train = train_num

In [ ]:
df_train['X0'] = train['X0']
df_train['X1'] = train['X1']
df_train['X2'] = train['X2']
df_train['X3'] = train['X3']
df_train['X4'] = train['X4']
df_train['X5'] = train['X5']
df_train['X6'] = train['X6']
df_train['X8'] = train['X8']
df_train['y'] = train['y']
df_train

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df_train.corr(), annot=True)

In [ ]:
df_train =df_train.drop('X29', axis=1)
df_train =df_train.drop('X279', axis=1)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(df_train.corr(), annot=True)

In [ ]:
dummies_train = pd.get_dummies(df_train)

In [ ]:
dummies_train.var(axis=0)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.1)
sel.fit_transform(dummies_train)[:5]

In [ ]:
dummies_train.head()

In [ ]:
X = dummies_train.drop('y', axis=1)
y = dummies_train['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
from xgboost import XGBRegressor

xgbr = XGBRegressor()
xgbr.fit(X_train, y_train)

In [ ]:
y_prediction = xgbr.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
r2_score(y_train,y_prediction)

In [ ]:
mean_squared_error(y_train,y_prediction)

In [ ]:
y_prediction = xgbr.predict(X_test)
r2_score(y_test, y_prediction)

In [ ]:
mean_squared_error(y_test,y_prediction)

In [ ]:
df_test = test[['X127', 'X314', 'X261', 'X313', 'X263', 'X136','X232', 'X76']]

In [ ]:
df_test['X0'] = test['X0']
df_test['X1'] = test['X1']
df_test['X2'] = test['X2']
df_test['X3'] = test['X3']
df_test['X4'] = test['X4']
df_test['X5'] = test['X5']
df_test['X6'] = test['X6']
df_test['X8'] = test['X8']

In [ ]:
df_test

In [ ]:
df_test = pd.get_dummies(df_test)
df_test

In [ ]:
sel.fit_transform(df_test)[:5]

In [ ]:
y_pred = xgbr.predict(X_test)

In [ ]:
prediction = pd.DataFrame(y_pred)
prediction.head()